In [1]:
#svm vaja
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import datasets
from sklearn import svm
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')
import os

In [14]:

df = pd.read_csv(r'data\credit\UCI_Credit_Card.csv')
df.drop("ID", axis=1, inplace=True)
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000.0,1,3,1,39,0,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,150000.0,1,3,2,43,-1,-1,-1,-1,0,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,30000.0,1,2,2,37,4,3,2,-1,0,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,80000.0,1,3,1,41,1,-1,0,0,0,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


In [15]:
df.dtypes

LIMIT_BAL                     float64
SEX                             int64
EDUCATION                       int64
MARRIAGE                        int64
AGE                             int64
PAY_0                           int64
PAY_2                           int64
PAY_3                           int64
PAY_4                           int64
PAY_5                           int64
PAY_6                           int64
BILL_AMT1                     float64
BILL_AMT2                     float64
BILL_AMT3                     float64
BILL_AMT4                     float64
BILL_AMT5                     float64
BILL_AMT6                     float64
PAY_AMT1                      float64
PAY_AMT2                      float64
PAY_AMT3                      float64
PAY_AMT4                      float64
PAY_AMT5                      float64
PAY_AMT6                      float64
default.payment.next.month      int64
dtype: object

In [27]:
df["EDUCATION"].unique()
len(df.loc[(df["EDUCATION"] == 0)|(df["MARRIAGE"] == 0)])
len(df)

30000

In [34]:
df_no_missing = df.loc[(df["EDUCATION"] != 0) & (df["MARRIAGE"] != 0)]
len(df_no_missing)


numpy.ndarray

In [68]:
#len(df.loc[df.columns[-1]] == 0)
df_no_default = df_no_missing.loc[df_no_missing[df_no_missing.columns[-1]] == 0]
df_default = df_no_missing.loc[df_no_missing[df_no_missing.columns[-1]] == 1]
df_no_default_downsample = df_no_default.sample(replace=False,
                                    n=1000,
                                    random_state=42)
df_default_downsample = df_default.sample(replace=False,
                                    n=1000,
                                    random_state=42)

In [69]:
df_downsample = pd.concat((df_no_default_downsample, df_default_downsample))


In [70]:
X = df_downsample[df_downsample.columns[:-1]]
y = df_downsample[df_downsample.columns[-1]]

In [71]:
#kategorizacija ustrezniz atributov
X_encoded = pd.get_dummies(X, columns=[
    "SEX", "EDUCATION", "MARRIAGE",
    "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"
])


In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=42)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
from sklearn.preprocessing import scale
X_test_scaled = scale(X_test)


In [94]:
from sklearn.svm import SVC
clf = SVC(random_state=42)
clf.fit(X_train_scaled, y_train)


SVC(random_state=42)

In [95]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
#y_train_pred = cross_val_predict(clf, X_train_scaled, y_train, cv=3)
y_test_pred = clf.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_test_pred)
disp = ConfusionMatrixDisplay(cm)

cm

array([[201,  56],
       [ 95, 148]], dtype=int64)

In [104]:
param = [
    {"C" : (0.5, 1, 10, 100),
     "gamma" : ("scale", 1, 0.1, 0.01, 0.001, 0.0001),
     "kernel" :["rbf"]
     }]

In [105]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(SVC(), param, cv=5, scoring="accuracy", verbose=4)

In [106]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END ....C=0.5, gamma=scale, kernel=rbf;, score=0.663 total time=   0.1s
[CV 2/5] END ....C=0.5, gamma=scale, kernel=rbf;, score=0.693 total time=   0.2s
[CV 3/5] END ....C=0.5, gamma=scale, kernel=rbf;, score=0.710 total time=   0.1s
[CV 4/5] END ....C=0.5, gamma=scale, kernel=rbf;, score=0.687 total time=   0.1s
[CV 5/5] END ....C=0.5, gamma=scale, kernel=rbf;, score=0.713 total time=   0.1s
[CV 1/5] END ........C=0.5, gamma=1, kernel=rbf;, score=0.497 total time=   0.1s
[CV 2/5] END ........C=0.5, gamma=1, kernel=rbf;, score=0.543 total time=   0.1s
[CV 3/5] END ........C=0.5, gamma=1, kernel=rbf;, score=0.533 total time=   0.1s
[CV 4/5] END ........C=0.5, gamma=1, kernel=rbf;, score=0.560 total time=   0.1s
[CV 5/5] END ........C=0.5, gamma=1, kernel=rbf;, score=0.550 total time=   0.1s
[CV 1/5] END ......C=0.5, gamma=0.1, kernel=rbf;, score=0.617 total time=   0.1s
[CV 2/5] END ......C=0.5, gamma=0.1, kernel=rbf

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': (0.5, 1, 10, 100),
                          'gamma': ('scale', 1, 0.1, 0.01, 0.001, 0.0001),
                          'kernel': ['rbf']}],
             scoring='accuracy', verbose=4)

In [108]:
grid.best_score_
grid.best_estimator

SVC(C=100, gamma=0.001)

In [111]:
clf1 = SVC(kernel="rbf", C=100, gamma =0.001)
clf1.fit(X_train_scaled, y_train)
y_test_pred = clf1.predict(X_test_scaled)
confusion_matrix(y_test,y_test_pred )

array([[205,  52],
       [ 96, 147]], dtype=int64)